In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-for-oceanography/sample2.csv
/kaggle/input/ml-for-oceanography/test1.csv
/kaggle/input/ml-for-oceanography/train.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/ml-for-oceanography/train.csv')
test_data=pd.read_csv('/kaggle/input/ml-for-oceanography/test1.csv')
train_data.head()

,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m)
0,2017424150,2017,4,24,1,50,23,3.0,3.6,4.17,3.33,349,1019.1,4.7,5.1,4.2,0.39
1,2017424250,2017,4,24,2,50,30,2.9,3.3,5.26,4.21,358,1018.8,4.6,4.8,4.1,0.65
2,2017424350,2017,4,24,3,50,360,2.2,2.5,5.88,4.79,11,1018.3,4.6,4.7,4.2,0.90
3,2017424450,2017,4,24,4,50,355,2.5,3.1,5.88,4.95,23,1018.1,4.6,4.6,4.2,0.91
4,2017424550,2017,4,24,5,50,355,3.5,4.5,6.25,4.98,26,1018.3,4.5,4.6,4.0,0.88


creating some features

In [3]:
# train_data['WPOW']= 0.5 * 1025 * 9.81 * train_data['WVHT(m)'] * train_data['WVHT(m)'] * train_data['DPD(sec)']

In [4]:
train_data.head()

,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m)
0,2017424150,2017,4,24,1,50,23,3.0,3.6,4.17,3.33,349,1019.1,4.7,5.1,4.2,0.39
1,2017424250,2017,4,24,2,50,30,2.9,3.3,5.26,4.21,358,1018.8,4.6,4.8,4.1,0.65
2,2017424350,2017,4,24,3,50,360,2.2,2.5,5.88,4.79,11,1018.3,4.6,4.7,4.2,0.90
3,2017424450,2017,4,24,4,50,355,2.5,3.1,5.88,4.95,23,1018.1,4.6,4.6,4.2,0.91
4,2017424550,2017,4,24,5,50,355,3.5,4.5,6.25,4.98,26,1018.3,4.5,4.6,4.0,0.88


In [5]:
df=train_data.copy()

calculating the sine and cosine: Instead of using the raw wind direction feature, it may be helpful to convert it into its sine and cosine components to capture any directional patterns in the wind that affect wave height.

In [6]:
# Calculate the sine and cosine of wind direction
df['WDIR_sin'] = np.sin(np.radians(df['WDIR(degT)']))
df['WDIR_cos'] = np.cos(np.radians(df['WDIR(degT)']))
test_data['WDIR_sin'] = np.sin(np.radians(test_data['WDIR(degT)']))
test_data['WDIR_cos'] = np.cos(np.radians(test_data['WDIR(degT)']))

In [7]:
df.head()

,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m),WDIR_sin,WDIR_cos
0,2017424150,2017,4,24,1,50,23,3.0,3.6,4.17,3.33,349,1019.1,4.7,5.1,4.2,0.39,3.907311e-01,0.920505
1,2017424250,2017,4,24,2,50,30,2.9,3.3,5.26,4.21,358,1018.8,4.6,4.8,4.1,0.65,5.000000e-01,0.866025
2,2017424350,2017,4,24,3,50,360,2.2,2.5,5.88,4.79,11,1018.3,4.6,4.7,4.2,0.90,-2.449294e-16,1.000000
3,2017424450,2017,4,24,4,50,355,2.5,3.1,5.88,4.95,23,1018.1,4.6,4.6,4.2,0.91,-8.715574e-02,0.996195
4,2017424550,2017,4,24,5,50,355,3.5,4.5,6.25,4.98,26,1018.3,4.5,4.6,4.0,0.88,-8.715574e-02,0.996195


In [8]:
df.shape

(3886, 19)

In [9]:
df.columns

Index(['ID', '#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR(degT)', 'WSPD(m/s)',
       'GST(m/s)', 'DPD(sec)', 'APD(sec)', 'MWD(degT)', 'PRES(hPa)',
       'ATMP(degC)', 'WTMP(degC)', 'DEWP(degC)', 'WVHT(m)', 'WDIR_sin',
       'WDIR_cos'],
      dtype='object')

In [10]:
y=df['WVHT(m)']
df=df.drop(columns=['WVHT(m)'])
X = df.iloc[:, [2, 4, *range(6, df.shape[1])]]

In [11]:
X.head()

,MM,hh,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WDIR_sin,WDIR_cos
0,4,1,23,3.0,3.6,4.17,3.33,349,1019.1,4.7,5.1,4.2,3.907311e-01,0.920505
1,4,2,30,2.9,3.3,5.26,4.21,358,1018.8,4.6,4.8,4.1,5.000000e-01,0.866025
2,4,3,360,2.2,2.5,5.88,4.79,11,1018.3,4.6,4.7,4.2,-2.449294e-16,1.000000
3,4,4,355,2.5,3.1,5.88,4.95,23,1018.1,4.6,4.6,4.2,-8.715574e-02,0.996195
4,4,5,355,3.5,4.5,6.25,4.98,26,1018.3,4.5,4.6,4.0,-8.715574e-02,0.996195


after all this crap time to model

In [12]:
from catboost import CatBoostRegressor
model=CatBoostRegressor()
model.fit(X,y)

Learning rate set to 0.050737
0:	learn: 0.3571549	total: 61ms	remaining: 1m
1:	learn: 0.3424260	total: 65ms	remaining: 32.4s
2:	learn: 0.3287441	total: 72ms	remaining: 23.9s
3:	learn: 0.3162028	total: 75.4ms	remaining: 18.8s
4:	learn: 0.3035610	total: 81.9ms	remaining: 16.3s
5:	learn: 0.2914925	total: 85.3ms	remaining: 14.1s
6:	learn: 0.2800740	total: 88.5ms	remaining: 12.6s
7:	learn: 0.2701216	total: 91.6ms	remaining: 11.4s
8:	learn: 0.2608825	total: 94.6ms	remaining: 10.4s
9:	learn: 0.2515466	total: 96.8ms	remaining: 9.58s
10:	learn: 0.2424595	total: 99.2ms	remaining: 8.92s
11:	learn: 0.2334152	total: 102ms	remaining: 8.36s
12:	learn: 0.2254061	total: 105ms	remaining: 7.96s
13:	learn: 0.2180962	total: 107ms	remaining: 7.55s
14:	learn: 0.2109644	total: 110ms	remaining: 7.2s
15:	learn: 0.2036714	total: 112ms	remaining: 6.88s
16:	learn: 0.1974578	total: 114ms	remaining: 6.61s
17:	learn: 0.1910494	total: 117ms	remaining: 6.37s
18:	learn: 0.1846794	total: 119ms	remaining: 6.15s
19:	learn:

In [13]:
X_test=test_data.iloc[:, [2, 4, *range(6, df.shape[1])]]
X_test.head()

,MM,hh,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WDIR_sin,WDIR_cos
0,7,22,340,6.7,8.2,2.35,2.37,177,1011.5,18.6,16.8,17.8,-0.342020,0.939693
1,7,23,15,2.3,5.5,2.60,2.44,336,1010.7,19.0,16.9,17.8,0.258819,0.965926
2,7,1,11,1.6,2.7,3.13,2.95,341,1010.6,17.9,16.7,17.3,0.190809,0.981627
3,7,10,21,4.1,4.9,2.86,2.59,8,1015.8,20.0,20.3,19.5,0.358368,0.933580
4,7,11,16,4.5,5.4,2.74,2.64,16,1016.5,20.1,20.3,19.6,0.275637,0.961262


making predictions

In [14]:
pred=model.predict(X_test)

In [15]:
output=pd.DataFrame({'Id':test_data.ID,'WVHT(m)':pred})
output.to_csv('submission.csv',index=False)
print("Your submission was successful")

Your submission was successful
